In [ ]:
# Saatkrähen-Auswertung Rostock

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap, FastMarkerCluster
from shapely import wkt
from sklearn.cluster import DBSCAN
import numpy as np
import random
import contextily as ctx

# --- 1. CSV laden ---
file_path = "Saatrkaehen_Rostock_2025-04-22.csv"  # <- ggf. anpassen

df = pd.read_csv(file_path, sep=";")

# --- 2. WKT-Geometrie umwandeln ---
df = df[df["wkt_geom"].apply(lambda x: isinstance(x, str))].copy()
df["geometry"] = df["wkt_geom"].apply(wkt.loads)

# --- 3. GeoDataFrame erstellen ---
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4839")
gdf_wgs84 = gdf.to_crs(epsg=4326)

# --- 4. Kuchendiagramm: Besetzt vs. Unbesetzt ---
status_counts = gdf_wgs84["Status"].value_counts()
plt.figure(figsize=(5, 5))
plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%')
plt.title("Neststatus")
plt.show()

# --- 5. Histogramm Koloniegröße ---
plt.figure(figsize=(6, 4))
sns.histplot(gdf_wgs84["Anzahl Individuen"], bins=range(1, gdf_wgs84["Anzahl Individuen"].max()+2), discrete=True)
plt.title("Koloniegrößen je Baum")
plt.xlabel("Individuen")
plt.ylabel("Anzahl Bäume")
plt.grid()
plt.show()

# --- 6. Heatmap (statisch) ---
plt.figure(figsize=(6, 5))
sns.kdeplot(x=gdf.geometry.x, y=gdf.geometry.y, fill=True, cmap="Reds", bw_adjust=0.5)
plt.title("Heatmap Saatkrähennester")
plt.xlabel("UTM Ost")
plt.ylabel("UTM Nord")
plt.show()

# --- 6b. Heatmap mit Basemap ---
fig, ax = plt.subplots(figsize=(8, 6))
sns.kdeplot(
    x=gdf.geometry.x,
    y=gdf.geometry.y,
    fill=True,
    cmap="Reds",
    bw_adjust=0.5,
    ax=ax,
    thresh=0.05
)
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
ax.set_title("Heatmap der Saatkrähennester (mit Stadtkarte)")
plt.xlabel("UTM Ost")
plt.ylabel("UTM Nord")
plt.tight_layout()
plt.show()

# --- 7. Clusteranalyse mit DBSCAN ---
coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))
db = DBSCAN(eps=50, min_samples=3).fit(coords)
gdf["Cluster"] = db.labels_

# --- 8. Clusterkarte (statisch, mit Basemap) ---
fig, ax = plt.subplots(figsize=(8, 6))
gdf.plot(ax=ax, column="Cluster", categorical=True, legend=True, markersize=25, cmap="tab10")
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
ax.set_title("Cluster der Saatkrähennester (mit Stadtkarte)")
plt.xlabel("UTM Ost")
plt.ylabel("UTM Nord")
plt.tight_layout()
plt.show()

# --- 9. Interaktive Karte mit Folium (schnellere Marker-Cluster-Version) ---
center = [gdf_wgs84.geometry.y.mean(), gdf_wgs84.geometry.x.mean()]
map_folium = folium.Map(location=center, zoom_start=14)

marker_data = [
    [point.y, point.x, f"Cluster: {cluster} | Status: {status} | Individuen: {anzahl}"]
    for point, cluster, status, anzahl in zip(
        gdf_wgs84.geometry, gdf["Cluster"], gdf["Status"], gdf["Anzahl Individuen"])
]

FastMarkerCluster(
    [[lat, lon] for lat, lon, _ in marker_data],
).add_to(map_folium)

map_folium.save("Saatkraehen_Karte_Interaktiv.html")
map_folium  # Zeige Karte direkt im Notebook
